In [1]:
import boto3
from double_click import echo

from nab3 import AWS as NabAWS,  Filter

"""
You should be able to work your way through almost any situation with just 2 classes and 10 methods.
You can take these to understand each service and script out all things AWS using nab3.

Each of those will be covered here
"""

"""
Class 1 is NabAWS (AWS)
    This is used as a base connection to retrieve any available service from AWS/boto3
"""
AWS = NabAWS(boto3.Session())  # Session is optional

In [2]:
"""
Method 1 is the staticmethod `AWS.service_operations`
    This returns a list of all AWS services supported by nab3
    These are used as attributes on the AWS class.
    For example, ecs_cluster is a supported nab3 service so to interface with it would be `AWS.ecs_cluster`.
    From here you can use the typical classmethods supported by the Service class (get or list)
"""

echo(AWS.service_options())

[
  "alarm",
  "app_scaling_policy",
  "asg",
  "ecs_cluster",
  "ecs_instance",
  "ecs_service",
  "ecs_task",
  "elasticache_cluster",
  "elasticache_node",
  "instance",
  "kafka_broker",
  "kafka_cluster",
  "launch_configuration",
  "load_balancer",
  "load_balancer_classic",
  "metric",
  "pricing",
  "rds_cluster",
  "rds_instance",
  "scaling_policy",
  "security_group"
]


In [3]:
"""
Method 2 is the classmethod `Service.list_params`
    This method provides each supported argument's name and type for the Service.list classmethod

Method 3 is the classmethod `Service.get_params`
    This method provides each supported argument's name and type for the Service.get classmethod
"""

# Examples of each
echo(AWS.ecs_instance.list_params())
echo(AWS.ecs_instance.get_params())

"""

Method 4 is the classmethod `Service.list`
    This method return all of the Service instances that match the provided params.
    If no params are provided, all Service instances are returned.

    Example:
        ecs_clusters = await AWS.ecs_cluster.list()

Method 5 is the classmethod `Service.get`
    This is used to retrieve a single instance of the AWS Service object.
    An exception will be raised if there is no match.

    Example using the optional param `with_related`:
        example_cluster = await AWS.ecs_cluster.get(name='example', with_related=['asg', 'services'])

Method 6 is `service_instance.methods`
    Returns a list of all of the service instance's methods

    Example:
        echo(example_cluster.methods())

Method 7 is `service_instance.fields`
    Returns a dict of all of the service instance's attributes.
    Structure: dict(attr_name=str(attr_type))

    Example:
        echo(example_cluster.fields())

Method 8 is `service_instance.fetch`
    Retrieves services related to the current instance. Supports nested calls using __
    Example:
        ecs_clusters = await AWS.ecs_cluster.list()
        await ecs_clusters.fetch('asg__scaling_policies', 'services')
        # Now each cluster in ecs_clusters contains all of its services represented as an ECSService instance

Method 9 is `service_instance.load`
    Retrieves the object from AWS and loads it into the current instance.
    This is not a method that will be used very often as it is far less practical than `service_instance.fetch`.
    However, it felt like it was worth mentioning.

    Example:
        example_asg = await AWS.asg.get(name='example')
        example_asg.launch_configuration.load()
"""

[
  {
    "name": "cluster",
    "type": "<class 'str'>"
  },
  {
    "name": "filter",
    "type": "<class 'str'>"
  },
  {
    "name": "status",
    "type": "<class 'str'>"
  }
]
[
  {
    "name": "cluster",
    "type": "<class 'str'>"
  },
  {
    "name": "id",
    "type": "<class 'list'>"
  },
  {
    "name": "include",
    "type": "<class 'list'>"
  }
]


In [5]:
"""
Class 2 is `Filter`
    Filter provides a class to easily filter service objects using common operations.

    The decision to have a filter class and a run method is to make the process of hitting AWS more explicit.
    It's also worth mentioning that lookups to AWS can get costly fast and this helps to mitigate that.
    This is also why there's no filter method Service class.
    nab3 is designed so all necessary data is pulled at once and manipulated through out the script.

    For example, if you wanted to get ECS Cluster stats grouped by dev, stg, and prod:

    ecs_clusters = await AWS.ecs_cluster.list()
    f_prod = Filter(name__icontains_any=['prod-', 'production'])
    f_stg = Filter(name__icontains_any=['stg-', 'staging'])
    f_dev = Filter(name__icontains_any=['dev-', 'development'])
    prod_clusters = await f.run(ecs_clusters)
    stg_clusters = await f.run(ecs_clusters)
    dev_clusters = await f.run(ecs_clusters)

    # If this was an allowed method, the unassuming eye would think these were the same.
    # Surprise! The number of lookups is 3n (if there were a dev and stg equivalent for each prod cluster)
    prod_clusters = await AWS.ecs_cluster.filter(name__icontains_any=['prod-', 'production'])
    stg_clusters = await AWS.ecs_cluster.filter(name__icontains_any=['stg-', 'staging'])
    dev_clusters = await AWS.ecs_cluster.filter(name__icontains_any=['dev-', 'development'])

    Why? Because these evaluations are done as part of nab3 and are not supported by boto3 or the AWS API.

    To inspect nested services use __. For example, to get a list of all ecs clusters with an api service:
    await ecs_clusters.fetch('services')
    api_filter = Filter(services__name__icontains_any=['api'])
    api_clusters = await api_filter.run(ecs_clusters)

Method 10 is the staticmethod `Filter.operations`
    Contains a list of every operation supported in Filter().run
"""

echo(Filter.operations())

[
  "contains",
  "contains_all",
  "contains_any",
  "exact",
  "exact_all",
  "exact_any",
  "gt",
  "gte",
  "icontains",
  "icontains_all",
  "icontains_any",
  "iexact",
  "iexact_all",
  "iexact_any",
  "lt",
  "lte",
  "re",
  "re_all",
  "re_any"
]
